## **01. 헤드라인**

In [1]:
import requests 
from bs4 import BeautifulSoup
import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
import pprint
import re
import pandas as pd
import urllib3
urllib3.disable_warnings()

#### **언론사별 신문 1면 헤드라인**
* **5개 일간지가 2023년 5월 1일부터 12일까지의 평일 열흘 동안 발행한 신문 1면의 기사 제목**

In [27]:
press = {"한겨레": "028", '경향신문': '032', '한국일보': '469', '동아일보': '020', '조선일보': '023'} 
publicated_day = ['20230501', '20230502', '20230503', '20230504', '20230505',
                  '20230508', '20230509', '20230510', '20230511', '20230512']

headline = {}

for day in publicated_day:
    day_headline = {}
    
    for press_name, press_code in press.items():
        url = "https://media.naver.com/press/{}/newspaper?date={}".format(press_code, day)
        response = requests.get(url)
        bs = BeautifulSoup(response.text, "html.parser")
        press_headlines = []
        link = []
        
        for pre in bs.find_all(class_="newspaper_brick_item _start_page"):
            for head in pre.find_all(class_="newspaper_wrp type_main"):
                for li in head.find_all("li"):
                    title = re.sub(r"\n+", "", li.a.text)
                    press_headlines.append(title)
                    title_url = li.a.get("href")
                    link.append(title_url)
        
        day_headline[press_name] = {'headlines': press_headlines,
                                    'links': link}

    headline[day] = day_headline

    for day, day_headline in headline.items():
        for press_name in day_headline:
            if not day_headline[press_name]['headlines']:
                url = "https://media.naver.com/press/{}/newspaper?date={}".format(press[press_name], day)
                response = requests.get(url)
                bs = BeautifulSoup(response.text, "html.parser")
                press_headlines = []
                link = []
                
                pre = bs.find(class_="newspaper_brick_item _start_page")
                head = pre.find(class_="newspaper_wrp")
                for li in head.find_all("li"):
                    title = re.sub(r"\n+", "", li.a.text)
                    press_headlines.append(title)
                    title_url = li.a.get("href")
                    link.append(title_url)
                
                day_headline[press_name] = {'headlines': press_headlines,
                                    'links': link}


In [6]:
pprint.pprint(headline['20230512'])

{'경향신문': {'headlines': ['제사 주재 우선권, 장남 아닌 나이순',
                        '3년 넘게, 고생하셨습니다…정부 “코로나 비상사태 끝”',
                        '4년 만에 구제역 비상 …다시 ‘살처분’ 비명',
                        '구글 챗봇 ‘바드’, 한국어 탑재하고 오픈'],
          'links': ['https://n.news.naver.com/article/newspaper/032/0003223007?date=20230512',
                    'https://n.news.naver.com/article/newspaper/032/0003223009?date=20230512',
                    'https://n.news.naver.com/article/newspaper/032/0003223008?date=20230512',
                    'https://n.news.naver.com/article/newspaper/032/0003223021?date=20230512']},
 '동아일보': {'headlines': ['은행-대부업 퇴짜… 자영업자, ‘이자 1100%’ 사채 쓰기도',
                        '[단독]한전-가스公 ‘전직원 임금 동결 추진’ 자구책 오늘 발표',
                        '석달간 재정적자 54조… 年전망치 90% 넘어',
                        '대법 “제사 주재자, 장남 아니어도 된다”',
                        '코로나 1207일 만에 ‘엔데믹’ 선언… 의료진에 기립 박수'],
          'links': ['https://n.news.naver.com/article/newspaper/020/0003496831?date=20230512',
                    '

#### **언론사별 2주간의 조회수 TOP5 기사의 헤드라인**
* **5개 일간지가 2023년 5월 1일부터 12일까지의 평일 열흘 동안 지면과 온라인으로 발행했던 기사 중 조회수가 높은 5개 기사 목록**

In [11]:
service = ChromeService(executable_path=ChromeDriverManager().install())
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")
driver = webdriver.Chrome(service=service, options=options)

rankingnews = {}

for day in publicated_day:
    try:
        url = "https://news.naver.com/main/ranking/popularDay.naver?date={}".format(day)
        driver.get(url)

        for _ in range(5):
            driver.find_element(By.CSS_SELECTOR, ".button_rankingnews_more").click()

        office_cards = "#wrap > div.rankingnews._popularWelBase._persist > div.rankingnews_box_wrap._popularRanking > div._officeCard._officeCard{}"

        day_rankingnews = {}
        for num in range(0, 73, 12): 
            office_card = driver.find_elements(By.CSS_SELECTOR, office_cards.format(num))
            for rankingnews_box in office_card:
                for i in range(1, 13):
                    newspaper = rankingnews_box.find_element(By.CSS_SELECTOR, f"div:nth-child({i}) > a > strong")
                    if newspaper.text in press:
                        if newspaper.text not in day_rankingnews:
                            day_rankingnews[newspaper.text] = {'headlines': [],
                                                               'links': []}
                        for k in range(1, 6):
                            headline = rankingnews_box.find_element(By.CSS_SELECTOR, f"div:nth-child({i}) > ul > li:nth-child({k}) > div > a")
                            day_rankingnews[newspaper.text]['headlines'].append(headline.text)
                            title_url = headline.get_attribute("href")
                            day_rankingnews[newspaper.text]['links'].append(title_url)
        
    except Exception as e:
        print(f"Error on {day}")
        
    rankingnews[day] = day_rankingnews
    
driver.close()
driver.quit()


In [13]:
pprint.pprint(rankingnews['20230512'])

{'경향신문': {'headlines': ['“그간 고마웠다” 전화 한 통화에…달려간 신입 공무원 ‘극단 선택’ 막았다',
                        '‘제사는 누가 지내나’ 질문에 정실 맏아들 → 장남 → 맏이…30여년 걸쳐 바뀐 대법 판례',
                        '테라 권도형 몬테네그로에서 첫 재판···재산 묻자 “아파트 1채”',
                        '웃돈 주고 사던 ‘중고 전기차’…반년 만에 시세 ‘뚝’ 이유는',
                        '음탕한 암사자, 폭압의 여왕 미어캣···다윈의 고정관념 부수는 ‘암컷들’[책과 삶]'],
          'links': ['https://n.news.naver.com/article/032/0003223171?ntype=RANKING',
                    'https://n.news.naver.com/article/032/0003223012?ntype=RANKING',
                    'https://n.news.naver.com/article/032/0003223129?ntype=RANKING',
                    'https://n.news.naver.com/article/032/0003222941?ntype=RANKING',
                    'https://n.news.naver.com/article/032/0003223182?ntype=RANKING']},
 '동아일보': {'headlines': ['방귀 잦고 냄새 독해져…대장에 문제가 생긴 걸까?[베스트 닥터의 베스트 건강법]',
                        '에버랜드 화재…나무 모형에 불 붙어 [청계천 옆 사진관]',
                        '“김남국, 또다른 게임코인 상장前 10억 매입… ‘마브렉스’ 투자”',
                        '“김남국

**표를 만들어 병합**

In [42]:
headline_data = []

for date, news in headline.items():
    for press, articles in news.items():
        titles = articles['headlines']
        title_links = articles['links']
        for i in range(5):
            if i < len(titles):
                headline_data.append([date, press, i+1, titles[i], title_links[i]])
            else:
                headline_data.append([date, press, i+1, 'None', 'None'])

headline_df = pd.DataFrame(headline_data, columns=['day', 'press', 'index', 'headline', 'headline_link'], index=None)
headline_df

,day,press,index,headline,headline_link
0,20230501,한겨레,1,기업 이익도 한반도 미래도 못 챙긴 윤 대통령의 ‘가치외교’,https://n.news.naver.com/article/newspaper/028...
1,20230501,한겨레,2,[단독] 채점자를 과외교사로?…수상한 가덕도 신공항 자문단,https://n.news.naver.com/article/newspaper/028...
2,20230501,한겨레,3,None,None
3,20230501,한겨레,4,None,None
4,20230501,한겨레,5,None,None
...,...,...,...,...,...
245,20230512,조선일보,1,"장훈 “내 조국이니 말할수 있다...日에 사과하라, 돈내라 언제까지 할건가”",https://n.news.naver.com/article/newspaper/023...
246,20230512,조선일보,2,부실 여론조사 난립 막는다,https://n.news.naver.com/article/newspaper/023...
247,20230512,조선일보,3,"임금 동결, 한전공대 출연금 축소… 한전 자구안 오늘 발표",https://n.news.naver.com/article/newspaper/023...
248,20230512,조선일보,4,함께 코로나 극복한 영웅들...정치가 ‘간호법’으로 갈라놓았다,https://n.news.naver.com/article/newspaper/023...


In [46]:
ranking_data = []

for date, news in rankingnews.items():
    for press, articles in news.items():
        titles = articles['headlines']
        title_links = articles['links']
        if len(titles) == 0:
            ranking_data.append([date, press, 'None', 'None', 'None'])
        else:
            for i, article in enumerate(titles):
                ranking_data.append([date, press, i+1, titles[i], title_links[i]])

ranking_df = pd.DataFrame(ranking_data, columns=['day', 'press', 'ranking', 'rankingnews', 'rankingnews_link'], index=None)
ranking_df

,day,press,ranking,rankingnews,rankingnews_link
0,20230501,경향신문,1,전세 거래 올 스톱…“잘 살던 세입자도 방 빼겠다고 해”,https://n.news.naver.com/article/032/000322062...
1,20230501,경향신문,2,"유승민 “윤 대통령, 아메리칸 파이 키웠는지 몰라도 코리안 파이는?”",https://n.news.naver.com/article/032/000322073...
2,20230501,경향신문,3,"홍준표, 대구도축장 폐쇄에…경북 양돈농가 “대안도 없이, 다 죽으란 얘기냐”[현장에서]",https://n.news.naver.com/article/032/000322080...
3,20230501,경향신문,4,"[단독]SG증권발 폭락주, 가장 많이 보유한 정치인·공직자 누구?",https://n.news.naver.com/article/032/000322055...
4,20230501,경향신문,5,연산군이 질탕하게 놀았던 탕춘대…왜 유네스코 세계유산 감인가[이기환의 Hi-story],https://n.news.naver.com/article/032/000322068...
...,...,...,...,...,...
245,20230512,경향신문,1,“그간 고마웠다” 전화 한 통화에…달려간 신입 공무원 ‘극단 선택’ 막았다,https://n.news.naver.com/article/032/000322317...
246,20230512,경향신문,2,‘제사는 누가 지내나’ 질문에 정실 맏아들 → 장남 → 맏이…30여년 걸쳐 바뀐 대...,https://n.news.naver.com/article/032/000322301...
247,20230512,경향신문,3,테라 권도형 몬테네그로에서 첫 재판···재산 묻자 “아파트 1채”,https://n.news.naver.com/article/032/000322312...
248,20230512,경향신문,4,웃돈 주고 사던 ‘중고 전기차’…반년 만에 시세 ‘뚝’ 이유는,https://n.news.naver.com/article/032/000322294...


In [47]:
merged_df = pd.merge(ranking_df, headline_df, on=['day', 'press'], how='left')
merged_df = merged_df[merged_df['ranking'] == merged_df['index']]
merged_df.drop(['index'], axis=1, inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df

,day,press,ranking,rankingnews,rankingnews_link,headline,headline_link
0,20230501,경향신문,1,전세 거래 올 스톱…“잘 살던 세입자도 방 빼겠다고 해”,https://n.news.naver.com/article/032/000322062...,‘미국에 올인’ 후폭풍이 몰려온다,https://n.news.naver.com/article/newspaper/032...
1,20230501,경향신문,2,"유승민 “윤 대통령, 아메리칸 파이 키웠는지 몰라도 코리안 파이는?”",https://n.news.naver.com/article/032/000322073...,상처 입은 이들의 ‘아주 각별한 반상회’,https://n.news.naver.com/article/newspaper/032...
2,20230501,경향신문,3,"홍준표, 대구도축장 폐쇄에…경북 양돈농가 “대안도 없이, 다 죽으란 얘기냐”[현장에서]",https://n.news.naver.com/article/032/000322080...,None,None
3,20230501,경향신문,4,"[단독]SG증권발 폭락주, 가장 많이 보유한 정치인·공직자 누구?",https://n.news.naver.com/article/032/000322055...,None,None
4,20230501,경향신문,5,연산군이 질탕하게 놀았던 탕춘대…왜 유네스코 세계유산 감인가[이기환의 Hi-story],https://n.news.naver.com/article/032/000322068...,None,None
...,...,...,...,...,...,...,...
245,20230512,경향신문,1,“그간 고마웠다” 전화 한 통화에…달려간 신입 공무원 ‘극단 선택’ 막았다,https://n.news.naver.com/article/032/000322317...,"제사 주재 우선권, 장남 아닌 나이순",https://n.news.naver.com/article/newspaper/032...
246,20230512,경향신문,2,‘제사는 누가 지내나’ 질문에 정실 맏아들 → 장남 → 맏이…30여년 걸쳐 바뀐 대...,https://n.news.naver.com/article/032/000322301...,"3년 넘게, 고생하셨습니다…정부 “코로나 비상사태 끝”",https://n.news.naver.com/article/newspaper/032...
247,20230512,경향신문,3,테라 권도형 몬테네그로에서 첫 재판···재산 묻자 “아파트 1채”,https://n.news.naver.com/article/032/000322312...,4년 만에 구제역 비상 …다시 ‘살처분’ 비명,https://n.news.naver.com/article/newspaper/032...
248,20230512,경향신문,4,웃돈 주고 사던 ‘중고 전기차’…반년 만에 시세 ‘뚝’ 이유는,https://n.news.naver.com/article/032/000322294...,"구글 챗봇 ‘바드’, 한국어 탑재하고 오픈",https://n.news.naver.com/article/newspaper/032...


In [273]:
merged_df.to_csv("headline_ranking_news.csv", index=False)